In [1]:
import pandas as pd

In [2]:
from src.GWY_news_utils import aggregate_content,calculate_attitude_score,load_attitude_dictionary,regression_x_y

In [3]:
from statsmodels.iolib.summary2 import summary_col
import statsmodels.api as sm

In [4]:

def ols_same_x(y_list,x):
    
    result_list = []
    
    for y in y_list:
        
        X,Y = regression_x_y(y,x,y_predict_lag=1,y_include_lag=0,x_include_lag=0)
        model = sm.OLS(Y,X)
        result = model.fit()
        result_list.append(result)
    
    return result_list

In [5]:
def ols_same_y(y,x_list):
    
    result_list = []
    
    check = 0
    
    for x in x_list:
        
        if check == 0:
            X,Y = regression_x_y(y,x,y_predict_lag=1,y_include_lag=0,x_include_lag=0)
            model = sm.OLS(Y,X)
            result = model.fit()
            result_list.append(result)
            check = 1
            
        X,Y = regression_x_y(y,x,y_predict_lag=1,y_include_lag=1,x_include_lag=0)
        model = sm.OLS(Y,X)
        result = model.fit()
        result_list.append(result)
    
    return result_list

In [6]:
data_directory = "data/extra_index_return/"
df = pd.read_excel(data_directory+"index_category_selected_2.xls")
df["交易日期_TrdDt"] = pd.to_datetime(df["交易日期_TrdDt"])

In [7]:
df_wide = pd.pivot_table(df,values="收盘价()_ClPr",index=["交易日期_TrdDt"],columns=["指数代码_IdxCd"])

In [8]:
df_wide.iloc[:,0:5].to_csv(data_directory+"index_category_selected_2_wide.csv")

In [12]:
sum(df_wide.index > "2016-09-25")

1

In [15]:
df_wide[df_wide.index > "2016-08-27"]

指数代码_IdxCd,903,904,905,906,907,399903,399904,399905,399906,399907
交易日期_TrdDt,,,,,,,,,,
2016-08-29,3075.04,4208.15,6427.53,3854.44,5198.72,3075.04,4208.15,6427.53,NaN,NaN
2016-08-30,3083.55,4203.05,6425.75,3857.53,5195.03,3083.55,4203.05,6425.75,NaN,NaN
2016-08-31,3104.51,4208.87,6434.21,3871.89,5202.04,3104.51,4208.87,6434.21,NaN,NaN
2016-09-01,3079.92,4176.02,6381.48,3841.02,5160.34,3079.92,4176.02,6381.48,NaN,NaN
2016-09-02,3096.00,4181.89,6356.95,3846.72,5153.03,3096.00,4181.89,6356.95,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
2020-09-25,4607.61,5005.87,6236.89,4843.15,5575.73,4607.61,5005.87,6236.89,NaN,NaN
2020-09-28,4632.19,4993.38,6178.47,4841.63,5543.42,4632.19,4993.38,6178.47,NaN,NaN
2020-09-29,4631.74,5025.31,6228.77,4859.07,5583.50,4631.74,5025.31,6228.77,NaN,NaN


## Data Preparation

### Stock Return Data

In [54]:
data_directory = "data/extra_index_return/"

In [55]:
df = pd.read_csv(data_directory+"index_category_selected_2_wide.csv")

In [56]:
df["交易日期_TrdDt"] = pd.to_datetime(df["交易日期_TrdDt"])
df.index = df["交易日期_TrdDt"]
df = df.iloc[:,1:]
index_df = df

In [58]:
index_m = index_df.asfreq("BM",method="ffill")
index_m = index_m.pct_change()

index_w = index_df.asfreq("W",method="ffill")
index_w = index_w.pct_change()

index_d = index_df.asfreq("B",method="ffill")
index_d = index_d.pct_change()

In [59]:
index_d

,903,904,905,906,907
交易日期_TrdDt,,,,,
2010-01-04,NaN,NaN,NaN,NaN,NaN
2010-01-05,0.008151,0.008148,0.010409,0.008680,0.009262
2010-01-06,-0.008915,-0.000526,-0.001580,-0.005159,-0.001046
2010-01-07,-0.020005,-0.019486,-0.020027,-0.019882,-0.019752
2010-01-08,0.001686,0.004236,0.012401,0.004838,0.008261
...,...,...,...,...,...
2020-10-05,0.000000,0.000000,0.000000,0.000000,0.000000
2020-10-06,0.000000,0.000000,0.000000,0.000000,0.000000
2020-10-07,0.000000,0.000000,0.000000,0.000000,0.000000


### Government Index

In [60]:
gov_index_d = pd.read_csv("data/output/daily_score_df.csv")
gov_index_w = pd.read_csv("data/output/weekly_score_df.csv")
gov_index_m = pd.read_csv("data/output/monthly_score_df.csv")

In [61]:
gov_index_d.index = pd.to_datetime(gov_index_d["date"])
gov_index_w.index = pd.to_datetime(gov_index_w["date"])
gov_index_m.index = pd.to_datetime(gov_index_m["date"])

In [62]:
gov_index_d = gov_index_d.iloc[:,1:]
gov_index_w = gov_index_w.iloc[:,1:]
gov_index_m = gov_index_m.iloc[:,1:]

### Government Index with Control

In [63]:
gov_index_d_e = pd.read_csv("data/output/daily_score_df_extra.csv")
gov_index_w_e = pd.read_csv("data/output/weekly_score_df_extra.csv")
gov_index_m_e = pd.read_csv("data/output/monthly_score_df_extra.csv")

In [64]:
gov_index_d_e.index = pd.to_datetime(gov_index_d_e["date"])
gov_index_w_e.index = pd.to_datetime(gov_index_w_e["date"])
gov_index_m_e.index = pd.to_datetime(gov_index_m_e["date"])

In [65]:
gov_index_d_e = gov_index_d_e.iloc[:,1:]
gov_index_w_e = gov_index_w_e.iloc[:,1:]
gov_index_m_e = gov_index_m_e.iloc[:,1:]

## Regression Analysis

### Merge X and Y

In [66]:
merge_df_m = index_m.merge(gov_index_m,how="inner",right_index=True,left_index=True)
merge_df_w = index_w.merge(gov_index_w,how="inner",right_index=True,left_index=True)
merge_df_d = index_d.merge(gov_index_d,how="inner",right_index=True,left_index=True)

In [67]:
merge_df_m_e = index_m.merge(gov_index_m_e,how="inner",right_index=True,left_index=True)
merge_df_w_e = index_w.merge(gov_index_w_e,how="inner",right_index=True,left_index=True)
merge_df_d_e = index_d.merge(gov_index_d_e,how="inner",right_index=True,left_index=True)

### Set up the regression

#### MLS With No control

In [86]:
regression_df = merge_df_m

In [87]:
regression_df.columns

Index(['903', '904', '905', '906', '907', 'economy', 'reform', 'regulation',
       'intervention'],
      dtype='object')

In [88]:
regression_df

,903,904,905,906,907,economy,reform,regulation,intervention
2016-04-29,-0.013282,-0.029208,-0.027652,-0.021552,-0.028383,0.446322,0.924524,0.406472,0.398502
2016-05-31,0.005984,0.000640,-0.006394,0.001035,-0.003095,0.777191,2.393915,0.415610,0.315864
2016-06-30,-0.008658,0.001768,0.029622,0.005171,0.016610,1.340982,2.262460,0.286841,0.358551
2016-07-29,0.007629,0.029913,0.012966,0.014977,0.020740,1.135499,1.287650,0.245133,0.312755
2016-08-31,0.040951,0.034835,0.037292,0.038246,0.036156,0.272351,0.976909,0.272351,0.224985
2016-09-30,-0.019936,-0.026506,-0.016493,-0.020605,-0.021124,0.573587,0.868930,0.181869,0.236274
2016-10-31,0.027266,0.022549,0.019614,0.023715,0.020962,0.595705,1.405436,0.199757,0.253264
2016-11-30,0.072399,0.040184,0.020670,0.048397,0.029647,0.548715,1.224732,0.272163,0.300696
2016-12-30,-0.064865,-0.063905,-0.048887,-0.059817,-0.055860,0.440852,1.333861,0.310857,0.356073
2017-01-31,0.034041,0.004937,-0.006373,0.014570,-0.001163,0.387150,1.221010,0.327588,0.148904


In [89]:
#y_list_1 = [regression_df["16"],regression_df["10"],regression_df["9"]]
#y_list_2 = [regression_df["399330"],regression_df["399009"],
#            regression_df["399007"],regression_df["399010"],regression_df["399011"]]
y_list_3 = [regression_df["903"],regression_df["904"],regression_df["905"]]

x = regression_df.iloc[:,5:]

In [90]:
result_list = ols_same_x(y_list_3,x)

In [91]:
result_table = summary_col(result_list,
                             stars=True,
                             float_format="%0.3f",
                             info_dict={'N':lambda x: "{0:d}".format(int(x.nobs)),
                                        'R2':lambda x: "{:.2f}".format(x.rsquared)})

In [92]:
print(result_table.as_latex())

\begin{table}
\caption{}
\begin{center}
\begin{tabular}{lccc}
\hline
             &   903   &   904   &   905    \\
\midrule
const        & -0.031  & -0.020  & -0.015   \\
             & (0.026) & (0.032) & (0.032)  \\
economy      & 0.034   & 0.036   & 0.032    \\
             & (0.034) & (0.041) & (0.040)  \\
reform       & -0.008  & -0.010  & -0.014   \\
             & (0.018) & (0.021) & (0.021)  \\
regulation   & -0.024  & -0.035  & -0.020   \\
             & (0.050) & (0.060) & (0.059)  \\
intervention & 0.104** & 0.086   & 0.068    \\
             & (0.043) & (0.053) & (0.052)  \\
R-squared    & 0.059   & -0.002  & -0.022   \\
             & 0.131   & 0.076   & 0.056    \\
N            & 53      & 53      & 53       \\
R2           & 0.13    & 0.08    & 0.06     \\
\hline
\end{tabular}
\end{center}
\end{table}


#### SLS

In [282]:
regression_df = merge_df_m

In [283]:
y_list = [regression_df["9"],regression_df["10"],regression_df["16"],regression_df["300"],
         regression_df["399007"],regression_df["399009"],regression_df["399010"],regression_df["399011"]]
x = regression_df.iloc[:,13]

In [284]:
result_list = ols_same_x(y_list,x)

In [285]:
result_table = summary_col(result_list,
                             stars=True,
                             float_format="%0.3f",
                             info_dict={'N':lambda x: "{0:d}".format(int(x.nobs)),
                                        'R2':lambda x: "{:.2f}".format(x.rsquared)})

In [286]:
print(result_table.as_latex())

\begin{table}
\caption{}
\begin{center}
\begin{tabular}{lcccccccc}
\hline
             &    9    &    10   &    16   &   300   &  399007 &  399009 &  399010 &  399011  \\
\midrule
const        & -0.022  & -0.029* & -0.030* & -0.027  & -0.024  & -0.022  & -0.022  & -0.024   \\
             & (0.020) & (0.016) & (0.017) & (0.017) & (0.022) & (0.022) & (0.023) & (0.022)  \\
intervention & 0.059   & 0.089** & 0.095** & 0.085** & 0.077   & 0.057   & 0.051   & 0.069    \\
             & (0.044) & (0.037) & (0.038) & (0.040) & (0.049) & (0.049) & (0.052) & (0.049)  \\
R-squared    & 0.015   & 0.085   & 0.091   & 0.064   & 0.027   & 0.006   & -0.000  & 0.019    \\
             & 0.034   & 0.102   & 0.109   & 0.082   & 0.046   & 0.026   & 0.019   & 0.038    \\
N            & 53      & 53      & 53      & 53      & 53      & 53      & 53      & 53       \\
R2           & 0.03    & 0.10    & 0.11    & 0.08    & 0.05    & 0.03    & 0.02    & 0.04     \\
\hline
\end{tabular}
\end{center}
\end{table

### MLS with control 

In [319]:
regression_df.columns

Index(['9', '10', '16', '300', '399007', '399009', '399010', '399011',
       '399300', '399330', 'economy', 'reform', 'regulation', 'intervention',
       'total_length', 'number_of_text', 'EPU', 'EPU_change'],
      dtype='object')

In [428]:
regression_df = merge_df_m_e

In [457]:
y = regression_df["399011"]
x_list =[regression_df.iloc[:,10:14],regression_df.iloc[:,10:16],regression_df.iloc[:,10:]]

In [458]:
result_list = ols_same_y(y,x_list)

In [459]:
result_table = summary_col(result_list,
                             stars=True,
                             float_format="%0.3f",
                             info_dict={'N':lambda x: "{0:d}".format(int(x.nobs)),
                                        'R2':lambda x: "{:.2f}".format(x.rsquared)},
                          regressor_order=["const","economy","intervention","reform","regulation",
                                           "total_length","number_of_text","EPU","EPU_change"])

In [460]:
print(result_table.as_latex())

\begin{table}
\caption{}
\begin{center}
\begin{tabular}{lcccc}
\hline
                 & 399011 I & 399011\_y I & 399011\_y II & 399011\_y III  \\
\midrule
const            & -0.017   & -0.017      & 0.032        & 0.016          \\
                 & (0.034)  & (0.034)     & (0.042)      & (0.046)        \\
economy          & 0.020    & 0.020       & 0.045        & 0.069          \\
                 & (0.043)  & (0.043)     & (0.044)      & (0.048)        \\
intervention     & 0.084    & 0.079       & 0.042        & 0.051          \\
                 & (0.055)  & (0.055)     & (0.058)      & (0.059)        \\
reform           & -0.009   & -0.008      & -0.010       & -0.016         \\
                 & (0.022)  & (0.022)     & (0.022)      & (0.023)        \\
regulation       & -0.028   & -0.029      & -0.063       & -0.047         \\
                 & (0.063)  & (0.063)     & (0.064)      & (0.066)        \\
total\_length    &          &             & -0.000       & -0.000         